In [29]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
import plotly.graph_objects as go
from datetime import datetime
from mpl_toolkits.mplot3d import Axes3D

In [76]:
def calcular_tiempo_a_madurez(df):
    """
    Calcula el tiempo hasta la madurez desde la fecha actual hasta las fechas en la columna 'Fecha' del DataFrame dado.
    La función retorna una columna con los tiempos calculados.

    Args:
        df (pd.DataFrame): DataFrame que contiene una columna 'Fecha' con las fechas de expiración.

    Returns:
        pd.Series: Serie con los tiempos hasta la madurez en años.
    """
    df_temp = df.copy()  # Trabajar con una copia para evitar efectos secundarios en el DataFrame original
    df_temp['Fecha'] = pd.to_datetime(df_temp['Fecha'])
    fecha_base = pd.Timestamp(datetime.now().date())
    return (df_temp['Fecha'] - fecha_base).dt.days / 365.25

In [156]:
def preparar_datos(df, precio_subyacente, tipo_opcion):
    """
    Prepara los datos para la interpolación necesaria para crear una superficie de volatilidad.

    Args:
        df (pd.DataFrame): DataFrame con las columnas 'Fecha', 'Strike', 'Vol_call', y 'Vol_put'.
        precio_subyacente (float): Precio actual del activo subyacente para calcular el Moneyness.
        tipo_opcion (str): Tipo de opción, 'call' o 'put'.

    Returns:
        tuple: Contiene tres arrays numpy que representan la malla de Tiempo a Madurez (T),
               Moneyness (M) y los valores interpolados de Volatilidad Implícita (IV).
    """
    # Calcula la madurez y asigna a la columna 'Maturity'
    df['Maturity'] = calcular_tiempo_a_madurez(df)
    
    # Calcula el Moneyness
    df['Moneyness'] = df['Strike'] / precio_subyacente
    
    # Selecciona la volatilidad según el tipo de opción y asegura no trabajar en copias
    if tipo_opcion == 'call':
        df = df.dropna(subset=['Vol_call']).copy()
        df.loc[:, 'VolatilidadImplicita'] = df['Vol_call']
    elif tipo_opcion == 'put':
        df = df.dropna(subset=['Vol_put']).copy()
        df.loc[:, 'VolatilidadImplicita'] = df['Vol_put']
    
    # Agrupa por 'Maturity' y 'Moneyness' y calcula la media de la volatilidad implícita
    df_agrupado = df.groupby(['Maturity', 'Moneyness'], as_index=False)['VolatilidadImplicita'].mean()

    # Prepara los datos para la interpolación
    Maturity = np.linspace(df_agrupado['Maturity'].min(), df_agrupado['Maturity'].max(), 100)
    Moneyness = np.linspace(df_agrupado['Moneyness'].min(), df_agrupado['Moneyness'].max(), 100)
    T_grid, M_grid = np.meshgrid(Maturity, Moneyness)

    # Interpolación para crear la superficie de volatilidad
    IV_grid = griddata(
        (df_agrupado['Maturity'].values, df_agrupado['Moneyness'].values),
        df_agrupado['VolatilidadImplicita'].values,
        (T_grid, M_grid),
        method='linear'
    )

    return df_agrupado


In [157]:
df = pd.read_csv('volatilidades.csv')
precio_subyacente = 11123.70  # Este valor debe ser el precio actual del subyacente


In [159]:
df_3 = preparar_datos(df, precio_subyacente, 'call')

df_3

,Maturity,Moneyness,VolatilidadImplicita
0,0.005476,0.984385,0.157836
1,0.005476,0.986632,0.165471
2,0.005476,0.988880,0.174677
3,0.005476,0.991127,0.180386
4,0.005476,1.006859,0.194383
5,0.024641,0.993375,0.142317
6,0.043806,0.952920,0.166283
7,0.043806,0.961910,0.162239
8,0.043806,0.975395,0.152036
9,0.043806,0.979890,0.149103


In [117]:
T, M, IV = preparar_datos(df, precio_subyacente, 'call')

In [128]:
M

array([[0.83605275, 0.83605275, 0.83605275, ..., 0.83605275, 0.83605275,
        0.83605275],
       [0.83832291, 0.83832291, 0.83832291, ..., 0.83832291, 0.83832291,
        0.83832291],
       [0.84059306, 0.84059306, 0.84059306, ..., 0.84059306, 0.84059306,
        0.84059306],
       ...,
       [1.05625781, 1.05625781, 1.05625781, ..., 1.05625781, 1.05625781,
        1.05625781],
       [1.05852796, 1.05852796, 1.05852796, ..., 1.05852796, 1.05852796,
        1.05852796],
       [1.06079812, 1.06079812, 1.06079812, ..., 1.06079812, 1.06079812,
        1.06079812]])

In [22]:
def pintar_superficie(X, Y, Z):
    """
    Pinta una superficie 3D de volatilidad implícita.
    """
    fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)])
    fig.update_layout(
        title='Superficie de Volatilidad Implícita',
        scene=dict(
            xaxis_title='Tiempo a Madurez',
            yaxis_title='Moneyness',
            zaxis_title='Volatilidad Implícita',
        ),
        autosize=True
    )
    fig.show()

100